# ASHRAE - Great Energy Predictor III
### *How much energy will a building consume?*

## Entendimento do problema

### Enunciado

*In this competition, you’ll develop accurate models of metered building energy usage in the following areas: chilled water, electric, hot water, and steam meters. The data comes from over 1,000 buildings over a three-year timeframe. With better estimates of these energy-saving investments, large scale investors and financial institutions will be more inclined to invest in this area to enable progress in building efficiencies.*

### Dados disponibilizados

**train.csv**
> * building_id - Foreign key for the building metadata.
> * meter - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, hotwater: 3}. Not every building has all meter types.
> * timestamp - When the measurement was taken
> * meter_reading - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error. 

**building_meta.csv**
> * site_id - Foreign key for the weather files.
> * building_id - Foreign key for training.csv
> * primary_use - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
> * square_feet - Gross floor area of the building
> * year_built - Year building was opened
> * floorcount - Number of floors of the building

**weather[train/test].csv** - Weather data from a meteorological station as close as possible to the site.

> * site_id
> * air_temperature - Degrees Celsius
> * cloud_coverage - Portion of the sky covered in clouds, in oktas
> * dew_temperature - Degrees Celsius
> * precip_depth_1_hr - Millimeters
sea_level_pressure - Millibar/hectopascals
> * wind_direction - Compass direction (0-360)
> * wind_speed - Meters per second

**test.csv**

> * row_id - Row id for your submission file
> * building_id - Building id code
> * meter - The meter id code
> * timestamp - Timestamps for the test data period

### Métrica de avaliação

*Root Mean Squared Logarithmic Error*



In [18]:
from sklearn.metrics import mean_squared_log_error

def rmsle(y_test,predictions):
    score = np.sqrt(mean_squared_log_error(y_test, predictions))
    return score

## Preparação dos dados

### Importação das bibliotecas e arquivos externos a serem utilizados.

In [19]:
import pandas as pd 
import numpy as np
from functions import *

### Carga dos dados

In [20]:
path = "./Data/"

In [21]:
building_meta = pd.read_csv(path+"building_metadata.csv")
weather_train = pd.read_csv(path+"weather_train.csv")
#weather_test = pd.read_csv(path+"weather_test.csv")
train_set = pd.read_csv(path+"train.csv")
#test_set = pd.read_csv(path+"test.csv")

### Tratamento inicial dos dados

In [22]:
print("Size of {:>20s} {:>8} rows, {:>2} columns / Mem. usage: {:7.2f} Mb".format("train_set data:",train_set.shape[0],train_set.shape[1],train_set.memory_usage().sum()/1024**2))
print("Size of {:>20s} {:>8} rows, {:>2} columns / Mem. usage: {:7.2f} Mb".format("weather_train data:",weather_train.shape[0],weather_train.shape[1],weather_train.memory_usage().sum()/1024**2))
print("Size of {:>20s} {:>8} rows, {:>2} columns / Mem. usage: {:7.2f} Mb".format("building_meta data:",building_meta.shape[0],building_meta.shape[1],building_meta.memory_usage().sum()/1024**2))

Size of      train_set data: 20216100 rows,  4 columns / Mem. usage:  616.95 Mb
Size of  weather_train data:   139773 rows,  9 columns / Mem. usage:    9.60 Mb
Size of  building_meta data:     1449 rows,  6 columns / Mem. usage:    0.07 Mb


### Redução do uso de memória

Aplicação de script para redução de algarismos reduntantes para auxiliar na redução do uso de memória local.

In [23]:
train_set = reduce_mem_usage(train_set)
weather_train = reduce_mem_usage(weather_train)
building_meta = reduce_mem_usage(building_meta)

Mem. usage decreased to  289.19 Mb (53.1% reduction)
Mem. usage decreased to    3.07 Mb (68.1% reduction)
Mem. usage decreased to    0.03 Mb (60.3% reduction)


### Configuração de data e hora nas colunas *timestamp*

In [24]:
train_set["timestamp"] = pd.to_datetime(train_set["timestamp"], format='%Y-%m-%d %H:%M:%S')
weather_train["timestamp"] = pd.to_datetime(weather_train["timestamp"], format='%Y-%m-%d %H:%M:%S')

## Análise dos dados

### Prévia dos dataframes

In [25]:
train_set.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [26]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.000000,6.0,20.00000,NaN,1019.5,0.0,0.000000
1,0,2016-01-01 01:00:00,24.406250,NaN,21.09375,-1.0,1020.0,70.0,1.500000
2,0,2016-01-01 02:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000
3,0,2016-01-01 03:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609


In [27]:
building_meta.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


### Verificação de valores vazios e NaN

In [28]:
total = train_set.isnull().sum()
percent = (train_set.isnull().sum()/train_set.isnull().count()*100)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_data

,Total,Percent
building_id,0,0.0
meter,0,0.0
timestamp,0,0.0
meter_reading,0,0.0


In [29]:
total = weather_train.isnull().sum()
percent = (weather_train.isnull().sum()/weather_train.isnull().count()*100)
missing_weather_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_weather_data

,Total,Percent
site_id,0,0.000000
timestamp,0,0.000000
air_temperature,55,0.039350
cloud_coverage,69173,49.489529
dew_temperature,113,0.080845
precip_depth_1_hr,50289,35.979052
sea_level_pressure,10618,7.596603
wind_direction,6268,4.484414
wind_speed,304,0.217496


In [33]:
total = building_meta.isnull().sum()
percent = (building_meta.isnull().sum()/building_meta.isnull().count()*100)
missing_building_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_building_data

,Total,Percent
site_id,0,0.000000
building_id,0,0.000000
primary_use,0,0.000000
square_feet,0,0.000000
year_built,774,53.416149
floor_count,1094,75.500345


## Tratamento secundário dos dados

Codificação de artibutos categóricos, preenchimento de valores faltantes, etc.

In [30]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
# building_meta['primary_use'] = labelEncoder.fit_transform(building_meta['primary_use'])

In [31]:
full_train = pd.merge(pd.merge(train_set,building_meta,on='building_id'),weather_train,on=['site_id','timestamp'])

In [47]:
# full_train = full_train.drop('floor_count',1)
# full_train['precip_depth_1_hr'] = full_train['precip_depth_1_hr'].fillna(0.0)
# full_train['cloud_coverage'] = full_train['cloud_coverage'].fillna(0.0)

## Treino e Validação

In [1]:
from sklearn.model_selection import cross_val_score